In [7]:
import numpy as np
import gudhi
import mdtraj as md
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import *
from math import pi
import MDAnalysis as mda
from MDAnalysis.analysis.base import (AnalysisBase,
                                      AnalysisFromFunction,
                                      analysis_class)
import pandas as pd
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.cluster import spectral_clustering
import random

In [8]:
traj5 = md.load('normal100frame.pdb')
traj5 = md.Trajectory.superpose(traj5, traj5[0], frame=0, atom_indices=None, ref_atom_indices=None, parallel=True)

In [9]:
traj6 = md.load('ahelix100frame.pdb')
traj6 = md.Trajectory.superpose(traj6, traj6[0], frame=0, atom_indices=None, ref_atom_indices=None, parallel=True)

In [10]:
carbon_atoms = traj6.topology.select("element == 'C'")

In [11]:
num_frames = traj6.n_frames

In [12]:
distance_matrix = np.zeros((num_frames, num_frames))

for frame1 in range(num_frames):
    persistence_pairs_frame1 = []
    carbon_coordinates_frame1 = traj6.xyz[frame1, carbon_atoms, :]
    rips_complex_frame1 = gudhi.RipsComplex(points=carbon_coordinates_frame1)
    simplex_tree_frame1 = rips_complex_frame1.create_simplex_tree(max_dimension=3)
    persistence_frame1 = simplex_tree_frame1.persistence()
    persistence_pairs1 = simplex_tree_frame1.persistence_intervals_in_dimension(1)
    persistence_diagrams1 = []
    persistence_diagrams1.append(persistence_pairs1)
    print(f'{frame1}')
    for frame2 in range(frame1 + 1, num_frames):
        persistence_pairs_frame2 = []
        carbon_coordinates_frame2 = traj6.xyz[frame2, carbon_atoms, :]
        rips_complex_frame2 = gudhi.RipsComplex(points=carbon_coordinates_frame2)
        simplex_tree_frame2 = rips_complex_frame2.create_simplex_tree(max_dimension=3)
        persistence_frame2 = simplex_tree_frame2.persistence()
        persistence_pairs2 = simplex_tree_frame2.persistence_intervals_in_dimension(1)
        persistence_diagrams2 = []
        persistence_diagrams2.append(persistence_pairs2)
        bottleneck_distance = gudhi.bottleneck_distance(persistence_pairs1, persistence_pairs2)

        distance_matrix[frame1, frame2] = bottleneck_distance

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [14]:
np.save('bhairpin100H1.npy',distance_matrix)